In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from pyharmx import PolyHarmInterpolator
from mpl_toolkits.axes_grid1 import make_axes_locatable

Inputs

In [ ]:
# Interpolation
smoothing = 0.0
order = 3
# Domain length
l = 1.0
nb_pts = 100

Set device and dtype

In [ ]:
device = "cuda:0"
dtype = torch.double

In [ ]:
torch.set_default_device(device)
torch.set_default_dtype(dtype)

Generate 2D grid

In [ ]:
a = 2*np.pi/l
x = torch.linspace(0, l, nb_pts)
y = torch.linspace(0, l, nb_pts)
xx, yy = torch.meshgrid(x, y, indexing='ij')

Compute exact solution

In [ ]:
u = torch.sin(a*xx) * torch.cos(3*a*yy)

Interpolate

In [ ]:
c = torch.cat([xx.reshape(1,-1,1), yy.reshape(1,-1,1)], dim=-1)
f = u.reshape(1,-1,1)

In [ ]:
intp = PolyHarmInterpolator(
  c=c,
  f=f,
  order=order,
  smoothing=smoothing,
  device=device,
  dtype=dtype
)
up = intp(c)

Evaluate interpolation error

In [ ]:
up = up.reshape(nb_pts, nb_pts)
ue = torch.abs(u - up)
emax = float(torch.max(ue))
print("Max error:", emax)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 9))
images = []
# Exact solution
im = ax[0].imshow(u.numpy(force=True), extent=[0,l,0,l])
ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
ax[0].set_title("Exact solution")
images.append(im)
# Interpolated solution
im = ax[1].imshow(up.numpy(force=True), extent=[0,l,0,l])
ax[1].set_xlabel("x")
ax[1].set_title("Interpolated solution")
images.append(im)
# Absolute error
im = ax[2].imshow(ue.numpy(force=True), extent=[0,l,0,l])
ax[2].set_xlabel("x")
ax[2].set_title("Absolute error")
images.append(im)
# Color bar
for i, im in enumerate(images):
  divider = make_axes_locatable(ax[i])
  cax = divider.append_axes('right', size='5%', pad=0.1)
  fig.colorbar(im, cax=cax, orientation='vertical')
fig.tight_layout()
plt.show()
plt.close()